In [1]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd

In [2]:
#Creating a database to connect to
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect() 

In [3]:
Base = declarative_base()
class Measurement(Base):
    __tablename__ = "measurements"
    id = Column(Integer, primary_key=True)
    station = Column(String(100))
    date = Column(Integer)
    prcp = Column(Integer)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
    

In [4]:
class Station(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key=True)
    station = Column(String(100))
    name = Column(String(100))
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [5]:
#creates tables in Database but not data
Base.metadata.create_all(engine) 

In [6]:
measurements = pd.read_csv("clean1_.csv", index_col=0)
measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.16,73


In [7]:
stations = pd.read_csv("clean2_.csv", index_col=0)
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In order to insert data into database it needs to be in a dictrionary

In [8]:
#convert to dict first
data_m = measurements.to_dict(orient='records')
data_s = stations.to_dict(orient='records')

In [9]:
#Creates a reflection or connection to insert data to database
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
#Save the reference to the `station` table as a variable
table_s = sqlalchemy.Table('stations', metadata, autoload=True) 
table_m = sqlalchemy.Table('measurements', metadata, autoload=True) 

In [12]:
conn.execute(table_s.delete()) 
conn.execute(table_m.delete()) 


In [13]:
#<<<< Magic happens, data is being inserted into tables>>>
conn.execute(table_s.insert(), data_s) 
conn.execute(table_m.insert(), data_m) 

In [14]:
conn.execute("select * from measurements").fetchall() 

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-06', 0.16, 73),
 (6, 'USC00519397', '2010-01-07', 0.06, 70),
 (7, 'USC00519397', '2010-01-08', 0, 64),
 (8, 'USC00519397', '2010-01-09', 0, 68),
 (9, 'USC00519397', '2010-01-10', 0, 73),
 (10, 'USC00519397', '2010-01-11', 0.01, 64),
 (11, 'USC00519397', '2010-01-12', 0, 61),
 (12, 'USC00519397', '2010-01-14', 0, 66),
 (13, 'USC00519397', '2010-01-15', 0, 65),
 (14, 'USC00519397', '2010-01-16', 0, 68),
 (15, 'USC00519397', '2010-01-17', 0, 64),
 (16, 'USC00519397', '2010-01-18', 0, 72),
 (17, 'USC00519397', '2010-01-19', 0, 66),
 (18, 'USC00519397', '2010-01-20', 0, 66),
 (19, 'USC00519397', '2010-01-21', 0, 69),
 (20, 'USC00519397', '2010-01-22', 0, 67),
 (21, 'USC00519397', '2010-01-23', 0, 67),
 (22, 'USC00519397', '2010-01-24', 0.01, 71),
 (23, 'USC00519397', '2010-01-25', 0, 67),
 (24,

In [15]:
conn.execute("select * from stations").fetchall() 

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]